In [1]:
from datetime import datetime
import time
import timeit
from functools import partial
import subprocess
import sys
# import cv2

# import cupy

import jax
import jax.numpy as jnp
import jax.scipy as jsp
# import jax.scipy as jsp
import numba as nb
from numba import jit, njit
from numba import int32, float64
from numba.typed import List
from numba.experimental import jitclass

import pygame as pg
import numpy as np
import scipy as sp
# import matplotlib.pyplot as plt
import typing as t

import os

pygame 2.3.0 (SDL 2.24.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class Model():

    def __init__(
            self, 
            # C,
            # c0, # : List(List()),
            # c1,
            # m : np.ndarray,
            # s : np.ndarray,
            # h : np.ndarray,
            # pos,
            # dt,
            # theta_A,
            g_func = jax.jit(lambda x, m, s: jnp.exp(-((x - m) / s)**2 / 2) * 2 - 1), # : t.Callable,
    ) -> None:
        # self.C = C
        # self.c0 = c0
        # self.c1 = c1
        # self.m = m
        # self.s = s
        # self.h = h
        # self.pos = pos
        # self.dt = dt
        # self.theta_A = theta_A
        self.g_func = g_func
        # self.compute_G = self

        self.G_func = self.compile_G_func()
        # self.gradient_func = self.compile_gradient_func()
        # self.mus_func = self.compile_mus_func()
        # self.flow_func = self.compile_flow_function()


    

    
    # @jax.jit
    def compile_G_func(
            self
            # g_func, # : t.Callable,
            # c2, # : List(List()),
            # m : np.ndarray,
            # s : np.ndarray,
            # h : np.ndarray
    ):

        # A = jnp.empty((2**8, 2**8, 3))
        #  IN LINUX, PASS ALL NP TO JNP
        def from_U_compute_H(U
                                # g_func, # : t.Callable,
                                # c2, # : List(List()),
                                # m : np.ndarray,
                                # s : np.ndarray,
                                # h : np.ndarray,
                                # C : int
                                ):

            return self.g_func(U + 2)
        
        # return jax.jit(f, static_argnames = ["A"])
        return jax.jit(from_U_compute_H)

In [5]:
func = lambda x: x * 3

model = Model(func)

res = model.G_func(2)

print(res)

12


In [89]:
world = np.ones((2, 2, 3)) * 0.5


k = np.array([
                [1., 0., -1.],
                [2., 0., -2.],
                [1., 0., -1.]
    ])

kx = np.array([
                [1., 0., -1.],
                [2., 0., -2.],
                [1., 0., -1.]
])
ky = np.transpose(kx)





def sobel(A, k2):
    return np.dstack([sp.signal.convolve2d(A[:, :, c], k2, mode = 'same') 
                    for c in range(A.shape[-1])])

def compute_gradient(H):
        
        # @jax.jit
    return np.concatenate((sobel(H, k.transpose())[:, :, None, :], sobel(H, k)[:, :, None, :]),
                            axis = 2)



def sobel_x(A):
  return np.dstack([sp.signal.convolve2d(A[:, :, c], kx, mode = 'same') 
                    for c in range(A.shape[-1])])
def sobel_y(A):
  return np.dstack([sp.signal.convolve2d(A[:, :, c], ky, mode = 'same') 
                    for c in range(A.shape[-1])])
  
# @jax.jit
def originalSobel(A):
  return np.concatenate((sobel_y(A)[:, :, None, :], sobel_x(A)[:, :, None, :]),
                         axis = 2)

print(world.shape)

(2, 2, 3)


In [90]:
origSob = originalSobel(world)
genSob = compute_gradient(world)
print(np.array_equal(origSob, genSob))
print(origSob.shape, genSob.shape)

True
(2, 2, 2, 3) (2, 2, 2, 3)


In [74]:
print(origSob, '\n\n\n')
print(genSob, '\n\n\n')

[[[[ 1.5  1.5  1.5]
   [ 1.5  1.5  1.5]]

  [[ 1.5  1.5  1.5]
   [-1.5 -1.5 -1.5]]]


 [[[-1.5 -1.5 -1.5]
   [ 1.5  1.5  1.5]]

  [[-1.5 -1.5 -1.5]
   [-1.5 -1.5 -1.5]]]] 



[[[[ 1.5  1.5  1.5]
   [ 1.5  1.5  1.5]]

  [[-1.5 -1.5 -1.5]
   [ 1.5  1.5  1.5]]]


 [[[ 1.5  1.5  1.5]
   [-1.5 -1.5 -1.5]]

  [[-1.5 -1.5 -1.5]
   [-1.5 -1.5 -1.5]]]] 





In [46]:
orig = np.array([[[[ 4.5],
         [ 4.5]],

        [[ 6. ],
         [ 0. ]],

        [[ 6. ],
         [ 0. ]],

        ...,

        [[ 6. ],
         [ 0. ]],

        [[ 6. ],
         [ 0. ]],

        [[ 4.5],
         [-4.5]]],


       [[[ 0. ],
         [ 6. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        ...,

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [-6. ]]],


       [[[ 0. ],
         [ 6. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        ...,

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [-6. ]]],


       ...,


       [[[ 0. ],
         [ 6. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        ...,

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [-6. ]]],


       [[[ 0. ],
         [ 6. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        ...,

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [-6. ]]],


       [[[-4.5],
         [ 4.5]],

        [[-6. ],
         [ 0. ]],

        [[-6. ],
         [ 0. ]],

        ...,

        [[-6. ],
         [ 0. ]],

        [[-6. ],
         [ 0. ]],

        [[-4.5],
         [-4.5]]]])

/tmp/ipykernel_45032/2077769786.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  orig = np.array([[[[ 4.5],


In [49]:
gen = np.array([[[[ 4.5],
         [ 4.5]],

        [[ 0. ],
         [ 6. ]],

        [[ 0. ],
         [ 6. ]],

        ...,

        [[ 0. ],
         [ 6. ]],

        [[ 0. ],
         [ 6. ]],

        [[-4.5],
         [ 4.5]]],


       [[[ 6. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        ...,

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[-6. ],
         [ 0. ]]],


       [[[ 6. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        ...,

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[-6. ],
         [ 0. ]]],


       ...,


       [[[ 6. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        ...,

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[-6. ],
         [ 0. ]]],


       [[[ 6. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        ...,

        [[ 0. ],
         [ 0. ]],

        [[ 0. ],
         [ 0. ]],

        [[-6. ],
         [ 0. ]]],


       [[[ 4.5],
         [-4.5]],

        [[ 0. ],
         [-6. ]],

        [[ 0. ],
         [-6. ]],

        ...,

        [[ 0. ],
         [-6. ]],

        [[ 0. ],
         [-6. ]],

        [[-4.5],
         [-4.5]]]])

/tmp/ipykernel_45032/426080914.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gen = np.array([[[[ 4.5],


In [52]:
np.array_equal(orig, gen)

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed